# IAA Laboratori 9 - Laboratory Report

In [1]:
%pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [51]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


from pandas import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
from statsmodels.genmod.generalized_linear_model import GLM
from pandas.plotting import scatter_matrix
from scipy.stats import boxcox
from collections import deque

In [13]:
import pandas as pd
import numpy as np

In [3]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
cirrhosis_patient_survival_prediction = fetch_ucirepo(id=878) 
  
# data (as pandas dataframes) 
X = cirrhosis_patient_survival_prediction.data.original 
y = cirrhosis_patient_survival_prediction.data.targets 
  
# metadata 
print(cirrhosis_patient_survival_prediction.metadata) 
  
# variable information 
print(cirrhosis_patient_survival_prediction.variables) 

{'uci_id': 878, 'name': 'Cirrhosis Patient Survival Prediction', 'repository_url': 'https://archive.ics.uci.edu/dataset/878/cirrhosis+patient+survival+prediction+dataset-1', 'data_url': 'https://archive.ics.uci.edu/static/public/878/data.csv', 'abstract': 'Utilize 17 clinical features for predicting survival state of patients with liver cirrhosis. The survival states include 0 = D (death), 1 = C (censored), 2 = CL (censored due to liver transplantation).', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular'], 'num_instances': 418, 'num_features': 17, 'feature_types': ['Real', 'Categorical'], 'demographics': ['Age', 'Sex'], 'target_col': ['Status'], 'index_col': ['ID'], 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1989, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5R02G', 'creators': ['E. Dickson', 'P. Grambsch', 'T. Fleming', 'L. Fisher', 'A. Langworthy'], 'intro_paper': {'title': 'Prognos

In [6]:
df = pd.DataFrame(X,  cirrhosis_patient_survival_prediction.original)


In [7]:
df.shape

(418, 20)

In [8]:
df.sample(10,random_state=42)

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
321,322,2011,D,NaN,23376,F,NaN,NaN,NaN,N,1.1,NaN,3.69,NaN,NaN,NaN,NaN,139,10.5,NaN
324,325,4795,C,NaN,12419,F,NaN,NaN,NaN,N,1.8,NaN,3.24,NaN,NaN,NaN,NaN,NaN,18.0,2.0
388,389,1581,C,NaN,24472,F,NaN,NaN,NaN,N,0.7,NaN,3.06,NaN,NaN,NaN,NaN,165,10.0,4.0
56,57,3282,D,D-penicillamine,19567,F,N,Y,N,S,2.3,260,3.18,231,11320.2,105.78,94,216,12.4,3.0
153,154,140,D,D-penicillamine,25340,M,N,N,Y,Y,2.4,168,2.56,225,1056.0,120.90,75,108,14.1,3.0
30,31,3839,D,Placebo,15177,F,N,Y,N,N,4.7,296,3.44,114,9933.2,206.40,101,195,10.3,2.0
72,73,4190,C,Placebo,14060,F,N,N,N,N,0.7,132,3.60,17,423.0,49.60,56,265,11.0,1.0
82,83,4050,C,D-penicillamine,20459,F,N,Y,N,S,1.3,250,3.50,48,1138.0,71.30,100,81,12.9,4.0
258,259,1615,C,Placebo,21904,F,N,Y,N,N,3.1,277,2.97,42,1110.0,125.00,126,221,9.8,3.0
416,417,691,C,NaN,21185,F,NaN,NaN,NaN,N,0.8,NaN,3.75,NaN,NaN,NaN,NaN,269,10.4,3.0


In [79]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   N_Days         418 non-null    float64
 1   Status         418 non-null    object 
 2   Drug           313 non-null    object 
 3   Age            418 non-null    float64
 4   Sex            418 non-null    object 
 5   Ascites        313 non-null    object 
 6   Hepatomegaly   313 non-null    object 
 7   Spiders        313 non-null    object 
 8   Edema          418 non-null    object 
 9   Bilirubin      316 non-null    float64
 10  Cholesterol    312 non-null    object 
 11  Albumin        409 non-null    float64
 12  Copper         312 non-null    object 
 13  Alk_Phos       269 non-null    float64
 14  SGOT           305 non-null    float64
 15  Tryglicerides  312 non-null    object 
 16  Platelets      411 non-null    object 
 17  Prothrombin    395 non-null    float64
 18  Stage     

In [19]:
unique_counts = pd.DataFrame.from_records([(col, df[col].nunique()) for col in df.columns],
                                         columns=['col_name', 'unique']).sort_values(by=['unique'])
unique_counts

,col_name,unique
5,Sex,2
9,Edema,3
8,Spiders,3
2,Status,3
3,Drug,3
7,Hepatomegaly,3
6,Ascites,3
19,Stage,4
18,Prothrombin,48
10,Bilirubin,98


In [ ]:
#Eliminate variable ID from database (does not provide info)
df.drop(['ID'], axis=1, inplace=True)

In [78]:
a = df.columns.tolist()
print(a)

Ncolumns = ['N_Days','Age','Bilirubin','Albumin','Alk_Phos','SGOT','Prothrombin','Stage']
Ccolumns = []
for elem in a:
    if elem in Ncolumns:
        pass
    else:
        Ccolumns.append(elem)
print(Ncolumns)
print(Ccolumns)



['N_Days', 'Status', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin', 'Stage']


'Status'

['N_Days', 'Age', 'Bilirubin', 'Albumin', 'Alk_Phos', 'SGOT', 'Prothrombin', 'Stage']
deque(['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Cholesterol', 'Copper', 'Tryglicerides', 'Platelets'])


In [85]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
N_Days,311.0,2012.681672,1119.522141,51.00,1201.50,1847.00,2702.50,4556.00
Age,311.0,18250.810289,3856.973289,9598.00,15392.50,18176.00,20696.00,28650.00
Bilirubin,234.0,1.318803,0.835729,0.30,0.70,1.10,1.80,3.40
Albumin,305.0,3.541902,0.376753,2.53,3.33,3.56,3.80,4.52
Alk_Phos,268.0,1275.451493,567.443503,289.00,822.25,1133.00,1646.25,2870.00
SGOT,305.0,117.945016,47.515459,26.35,79.98,113.15,147.25,246.45
Prothrombin,295.0,10.568475,0.736920,9.00,10.00,10.60,11.00,12.40
Stage,311.0,3.028939,0.877569,1.00,2.00,3.00,4.00,4.00


In [ ]:
df.describe(exclude = ['float', 'int64']).T

In [ ]:
#NaN
df.isna().sum().sort_values(ascending=False)
round(100* (df.isnull().sum() / len(df)).sort_values(ascending=False),2)

In [ ]:
#plot all variables in df separately
for elem in Ncolumns:
    plt.figure()
    df[elem].plot.hist(bins=20)
    plt.title(elem)
    plt.show()

In [ ]:
#plot all categorical variables
for elem in Ccolumns:
    plt.figure(figsize=(6,3), dpi=100)
    sns.countplot(data=df,x=elem)
    plt.legend(loc=(1.05,0.5))

In [ ]:
#study outliers with mean and quartiles
for elem in Ncolumns:
    print(elem)
    Q1 = df[elem].quantile(0.25)
    Q3 = df[elem].quantile(0.75)
    IQR = Q3 - Q1
    print('Q1 =',Q1,'Q3 =', Q3,'IQR =', IQR)
    lim_inf = Q1 - 1.5 * IQR
    lim_sup = Q3 + 1.5 * IQR

    outliers = df[(df[elem] < lim_inf) | (df[elem] > lim_sup)]
    print(outliers)
    outliers = outliers
    print("\n")
    #turn outliers into NaN
    
    print("\n")
    print("*"*70)
    print("\n")
df.describe().T

In [75]:
#turn outliers into NaN
for elem in Ncolumns:
    Q1 = df[elem].quantile(0.25)
    Q3 = df[elem].quantile(0.75)
    IQR = Q3 - Q1
    lim_inf = Q1 - 1.5 * IQR
    lim_sup = Q3 + 1.5 * IQR
    
    df.loc[(df[elem] < lim_inf) | (df[elem] > lim_sup), elem] = np.nan
df.describe().T

,count,mean,std,min,25%,50%,75%,max
N_Days,418.0,1917.782297,1104.672992,41.00,1092.75,1730.00,2613.50,4795.00
Age,418.0,18533.351675,3815.845055,9598.00,15644.50,18628.00,21272.50,28650.00
Bilirubin,316.0,1.312025,0.815279,0.30,0.70,1.10,1.80,3.40
Albumin,409.0,3.519022,0.387374,2.48,3.26,3.54,3.78,4.52
Alk_Phos,269.0,1273.330855,567.450779,289.00,820.00,1132.00,1644.00,2870.00
SGOT,305.0,117.945016,47.515459,26.35,79.98,113.15,147.25,246.45
Prothrombin,395.0,10.575190,0.715167,9.00,10.00,10.60,11.00,12.40
Stage,412.0,3.024272,0.882042,1.00,2.00,3.00,4.00,4.00


In [86]:
#we find how much missings per row we have and eliminate those rows with too much missings
df['missings'] = df.isnull().sum(axis=1)
df['missings'].value_counts()
df = df[df['missings'] < 4]
df = df.drop(['missings'], axis=1)
df.shape
df.info()

missings
0    195
1     87
2     26
3      3
Name: count, dtype: int64

(311, 19)

<class 'pandas.core.frame.DataFrame'>
Index: 311 entries, 0 to 311
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   N_Days         311 non-null    float64
 1   Status         311 non-null    object 
 2   Drug           311 non-null    object 
 3   Age            311 non-null    float64
 4   Sex            311 non-null    object 
 5   Ascites        311 non-null    object 
 6   Hepatomegaly   311 non-null    object 
 7   Spiders        311 non-null    object 
 8   Edema          311 non-null    object 
 9   Bilirubin      234 non-null    float64
 10  Cholesterol    311 non-null    object 
 11  Albumin        305 non-null    float64
 12  Copper         311 non-null    object 
 13  Alk_Phos       268 non-null    float64
 14  SGOT           305 non-null    float64
 15  Tryglicerides  311 non-null    object 
 16  Platelets      311 non-null    object 
 17  Prothrombin    295 non-null    float64
 18  Stage          

In [88]:
cat = df.select_dtypes(include = 'object')
cat.info()
print("\n")
print('*'*70)
print("\n")
num = df.select_dtypes(include = ['float64', 'int64'])
num.info()

<class 'pandas.core.frame.DataFrame'>
Index: 311 entries, 0 to 311
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Status         311 non-null    object
 1   Drug           311 non-null    object
 2   Sex            311 non-null    object
 3   Ascites        311 non-null    object
 4   Hepatomegaly   311 non-null    object
 5   Spiders        311 non-null    object
 6   Edema          311 non-null    object
 7   Cholesterol    311 non-null    object
 8   Copper         311 non-null    object
 9   Tryglicerides  311 non-null    object
 10  Platelets      311 non-null    object
dtypes: object(11)
memory usage: 29.2+ KB


**********************************************************************


<class 'pandas.core.frame.DataFrame'>
Index: 311 entries, 0 to 311
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N_Days       311 non-null    float64